In [1]:
# import libraries
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures

Read in data

In [2]:
# read test and train data, print shapes
test = pd.read_csv('datasets/test.csv', index_col=0)
train = pd.read_csv('datasets/train.csv', index_col=0)
train.shape, test.shape

((2051, 80), (878, 79))

Concatenate train and test for data cleaning & feature engineering

In [3]:
test['SalePrice'] = np.nan
train['dataset'] = 0
test['dataset'] = 1

In [4]:
train_and_test = [test, train]
df = pd.concat(train_and_test)

### Fill missing values

In [5]:
# iterate through columns will Na values
for col in [c for c in df.columns if df[c].isna().sum() > 0]:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna("None")  # fill na with 'none' in object columns
    else:
        df[col] = df[col].fillna(0)       # fill na with 0 in numerical columns

### Encode ordinal categories

In [6]:
# map over ordinals with standard scale:
standard_mapper =  {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None':0}
standard_cols = ['Exter Qual', 'Exter Cond', 'Bsmt Cond', 'Bsmt Qual',
               'Heating QC', 'Kitchen Qual', 'Fireplace Qu', 'Garage Qual',
               'Garage Cond', 'Pool QC']
for col in standard_cols:
    df[col] = df[col].replace(standard_mapper).astype('float')

# map over basement ordinals:
bsmtft_mapper =  { 'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6, }
bsmt_fincols = ['BsmtFin Type 1', 'BsmtFin Type 2']
for col in bsmt_fincols:
    df[col] = df[col].replace(bsmtft_mapper).astype('float')

# map over ordinals with unique scales:
other_mapper = {
    'Lot Shape': { 'None': 0, 'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3': 3, },
    'Utilities': { 'AllPub': 4, 'NoSewr': 3, 'NoSeWa': 2, 'ELO': 1, 'None': 0, },
    'Land Slope': { 'None': 0, 'Gtl': 0, 'Mod': 1, 'Sev': 2, },
    'Bsmt Exposure': { 'Gd': 5, 'Av': 3, 'Mn': 2, 'No': 0, 'None': 0, },
    'Electrical': { 'SBrkr': 4, 'FuseA': 3, 'FuseF': 2, 'FuseP': 1, 'Mix': 4, 'None': 0, },
    'Functional': { 'None': 0, 'Typ': 0, 'Min1': 2, 'Min2': 3, 'Mod': 4, 'Maj1': 6, 'Maj2': 7, 'Sev': 8, 'Sal': 10, },
    'Garage Finish': { 'Fin': 3, 'RFn': 2, 'Unf': 1, 'None': 0, },
    'Paved Drive': { 'Y': 3, 'P': 2, 'N': 1, 'None': 0, },
    'Fence': { 'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'None': 0, },}
for col, mapper in other_mapper.items():
    df[col] = df[col].replace(mapper).astype('float')

### Revert ordinals to categorical
(Save numerical versions)

In [7]:
# define ordinal columns
ordinals = bsmt_fincols + standard_cols + [k for k in other_mapper.keys()]
ordinals.append('SalePrice')

In [8]:
ordinals.remove('SalePrice')
#df[[col+'_num' for col in ordinals]] = df[ordinals]
df[ordinals] = df[ordinals].astype('str')

### Engineer a few custom features

Create feature of total porch size

In [9]:
porches = [col for col in df.columns if 'porch' in col.lower()]
df['Total Porch'] = 0
for col in porches:
    df['porch_sum'] = df['Total Porch'] + df[col]

Create feature of total SF

In [10]:
df['Total SF'] = df['Gr Liv Area'] + df['Total Bsmt SF'] + df['Total Porch'] + df['Garage Area']

In [11]:
df['total baths'] = df['Bsmt Full Bath'] + df['Full Bath'] + 2*df['Bsmt Half Bath'] + 2*df['Half Bath']

In [12]:
df['avgroomsize'] = df['Total SF'] = df['TotRms AbvGrd']

A few interaction features

strong_corrs = list(train.corr()['SalePrice'].abs().sort_values(ascending=False)[:5].index)
strong_corrs.remove('SalePrice')
strong_corrs =[c for c in strong_corrs if c in df.columns] + [c+'_num' for c in strong_corrs if c+'_num' in df.columns]
for feat in strong_corrs:
    df[feat+'**2'] = df[feat]**2
    for comp in [c for c in strong_corrs if c != feat]:
        df[feat+'*'+comp] = df[feat]*df[comp]
    strong_corrs.remove(feat)
        

In [13]:
df['Total SF*Overall Qual'] = df['Total SF']*df['Overall Qual']
df['Garage Cars*Overall Qual'] = df['Garage Cars']*df['Overall Qual']
df['Total Bsmt SF*Overall Qual'] = df['Total Bsmt SF']*df['Overall Qual']
df['1st Flr SF*Overall Qual'] = df['1st Flr SF']*df['Overall Qual']
df['Year Built*Overall Qual'] = df['Year Built']*df['Overall Qual']
df['Full Bath*Overall Qual'] = df['Full Bath']*df['Overall Qual']
df['Garage Area*Overall Qual'] = df['Garage Area']*df['Overall Qual']
df['Fireplaces*Overall Qual'] = df['Fireplaces']*df['Overall Qual']

df['Gr Liv Area*Garage Cars'] = df['Gr Liv Area']*df['Garage Cars']
df['Full Bath*Garage Cars'] = df['Gr Liv Area']*df['Garage Cars']



Log a few x-vars

In [14]:
x_to_log = [ 'Lot Area', 'Year Built', 'Year Remod/Add', 'Total SF', 'Gr Liv Area',
            '1st Flr SF', 'Total SF*Overall Qual', 'total baths', ]
for target in x_to_log:
    df[target] = df[target].apply(np.log)


### Convert specific numericals to categorical
(Save numeric versions)

In [15]:
# We want these numerical features to get dummified
categoricals = ['Yr Sold', 'MS SubClass', 'Overall Qual', 'Overall Cond', 'Full Bath', 'Half Bath',
 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Type']
df[[col+'_num' for col in categoricals]] = df[categoricals]
df[categoricals] = df[categoricals].astype('str')

In [16]:
# exploiting seasonality | credit to https://jackdry.com/house-prices-advanced-regression-techniques-kaggle
df["SinMoSold"] = np.sin(2 * np.pi * df["Mo Sold"] / 12)
df["CosMoSold"] = np.cos(2 * np.pi * df["Mo Sold"] / 12)
df = df.drop("Mo Sold", axis=1)

### Identify high zero counts

In [17]:
zeros = [col for col in df.columns if
         len(df[df[col] == 0]) > 0 and '_' not in col and 'Price' not in col]
zeros.remove('dataset')
print('{} columns have more than one zero'.format(len(zeros)))

29 columns have more than one zero


In [18]:
# create dummy columns to identify zero values 
for col in zeros :   
    df[col+'_is_zero'] = np.where(df[col] == 0, 1, 0)

### Get Dummies

In [19]:
df = pd.get_dummies(df, drop_first=True)

### Deal with outliers

df['Lot Frontage outliers'] = np.where(df['Lot Frontage']>=150, 1, 0)
df['BsmtFin SF 1 outliers'] = np.where(df['BsmtFin SF 1']>=3000, 1, 0)
df['Total Bsmt SF outliers'] = np.where(df['Total Bsmt SF']>=4000, 1, 0)
df['1st Flr SF outliers'] = np.where(df['1st Flr SF']>=3500, 1, 0)
df['Garage Cars outliers'] = np.where(df['Garage Cars']>=4, 1, 0)
df['Wood Deck SF outliers'] = np.where(df['Wood Deck SF']>=1000, 1, 0)
df['Open Porch SF outliers'] = np.where(df['Open Porch SF']>=280, 1, 0)
df['Enclosed Porch outliers'] = np.where(df['Enclosed Porch']>=320, 1, 0)
df['Screen Porch outliers'] = np.where(df['Screen Porch']>=430, 1, 0)
df['Lot Area outliers1'] = df['Lot Area'].map( lambda x: '1' if x > 20000 and x < 80000 else 0)
df['Lot Area outliers2'] = df['Lot Area'].map( lambda x: '1' if x > 80000 else 0)

### Recover train and test

In [20]:
X_train = df[df['dataset'] == 0]
X_test = df[df['dataset'] == 1]
X_train.drop(columns='dataset', inplace=True)
X_test.drop(columns='dataset', inplace=True)

/Users/jfitz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
y_train = X_train['SalePrice']
X_train = X_train[X_train._get_numeric_data().drop(columns='SalePrice').columns]
X_test = X_test[X_test._get_numeric_data().drop(columns='SalePrice').columns]

#### Log transform

In [22]:
y_train_log = y_train.apply(np.log)

In [23]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [32]:
model = LinearRegression()
cross_val_score(model, X_train, y_train_log, cv=5).mean

<function ndarray.mean>

In [25]:
#interaction = PolynomialFeatures(include_bias=False, )
#X_train_inter = interaction.fit_transform(X_train)
#X_test_inter = interaction.fit_transform(X_test)

In [26]:
model.fit(X_train, y_train_log)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
y_pred = model.predict(X_test)

In [28]:
X_test.reset_index(inplace=True)

In [29]:
# create output df
output = pd.DataFrame( {'Id': X_test['Id']} )
output['SalePrice'] = np.exp(y_pred)

output.head()

,Id,SalePrice
0,2658,105860.285337
1,2718,154110.993542
2,2414,211404.254708
3,1989,102050.028828
4,625,175687.419364


In [30]:
output.to_csv('submissions/submission_30a.csv', index=False)